# Import libraries

In [2]:
import pandas as pd
import numpy as np

# Read the dataset

In [3]:
pd.set_option('display.max_columns', None)
data = pd.read_csv(r'C:\Users\keert\Desktop\fifa21 raw data.csv',low_memory=False)


# Data Cleaning

In [4]:
fifa= data.copy()

In [71]:
fifa.sample(3)

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,Contract,Positions,Height,Weight,Preferred Foot,BOV,Best Position,Joined,Loan Date End,Value,Wage,Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
10079,205163,E. Huws,Emyr Huws,https://cdn.sofifa.com/players/205/163/21_60.png,http://sofifa.com/player/205163/emyr-huws/210006/,Wales,26,65,66,\n\n\n\nIpswich Town,2017 ~ 2021,"CM, CDM",185cm,78kg,Left,66,CAM,28-Jun-17,NaN,€825K,€4K,€1.1M,294,64,62,49,67,52,312,64,60,56,66,66,321,67,66,67,63,58,334,70,67,68,64,65,318,63,62,60,67,66,71,184,59,63,62,40,13,6,6,9,6,1803,385,3 ★,3★,High,Medium,1 ★,66,64,65,65,60,65,2
12453,241534,S. McLoughlin,Sean McLoughlin,https://cdn.sofifa.com/players/241/534/21_60.png,http://sofifa.com/player/241534/sean-mcloughli...,Republic of Ireland,23,63,73,\n\n\n\nHull City,2019 ~ 2022,CB,191cm,74kg,Left,65,CB,26-Jul-19,NaN,€1M,€2K,€1.1M,224,50,29,61,58,26,227,55,29,32,55,56,260,51,55,45,60,49,273,41,70,67,74,21,249,64,61,44,39,41,60,188,61,64,63,42,8,7,6,14,7,1463,319,3 ★,2★,Medium,Medium,1 ★,53,31,49,54,62,70,11
6824,233090,J. Torres,Joaquín Torres,https://cdn.sofifa.com/players/233/090/21_60.png,http://sofifa.com/player/233090/joaquin-torres...,Argentina,23,68,77,\n\n\n\nNewell's Old Boys,2014 ~ 2021,RM,167cm,70kg,Left,69,LM,01-Dec-14,NaN,€2.7M,€6K,€2.4M,262,56,53,42,64,47,292,75,48,42,60,67,404,81,89,91,61,82,321,61,74,67,57,62,241,40,41,63,58,39,63,93,31,30,32,44,7,10,12,7,8,1657,365,3 ★,3★,Medium,Medium,1 ★,85,56,59,74,34,57,2


## Club

In [63]:
fifa['Club'].dtype

dtype('O')

In [26]:
fifa['Club']= fifa['Club'].str.strip()
fifa['Club'].unique()

## Contract

In [74]:
fifa['Contract'].dtype

dtype('O')

In [25]:
fifa['Contract'].unique()

In [24]:
for index,row in fifa.iterrows():
    if "On Loan" in row['Contract'] or 'Free' in row['Contract']:
        print (row['Contract'])

SyntaxError: incomplete input (737217895.py, line 3)

In [77]:
def contract_info(contract):
    if contract == 'Free' or 'On Loan' in contract:
        start_date= np.nan
        end_date= np.nan
        contract_length= 0
    else:
        start_date, end_date = contract.split(' ~ ')
        start_year= int(start_date[:4])
        end_year= int(end_date[:4])
        contract_length= end_year - start_year
    return start_date, end_date, contract_length

# Apply fn to Contract column & create new columns

new_cols =['Contract Start', 'Contract End', 'Contract Length(years)']
new_data = fifa['Contract'].apply(lambda x: pd.Series(contract_info(x)))

for i in range(len(new_cols)):
    fifa.insert(loc=fifa.columns.get_loc('Contract')+1+i, column = new_cols[i], value=new_data[i])



In [81]:
fifa[['Contract','Contract Start', 'Contract End', 'Contract Length(years)']].sample(3)

,Contract,Contract Start,Contract End,Contract Length(years)
13159,"Jun 30, 2021 On Loan",NaN,NaN,0.0
14983,2020 ~ 2020,2020,2020,0.0
8552,2018 ~ 2021,2018,2021,3.0


In [82]:
#contract categories

def contract_category(contract):
    if contract =='Free':
        return 'Free'
    elif 'On Loan' in contract:
        return 'On Loan'
    else:
        return 'Contract'
    
#apply fn to Contract column, add Contract Category column

fifa.insert(loc=fifa.columns.get_loc('Contract')+1, column='Contract Category',
            value= fifa['Contract'].apply(contract_category))


In [83]:
fifa[['Contract','Contract Start', 'Contract End', 'Contract Length(years)','Contract Category']].sample(3)

,Contract,Contract Start,Contract End,Contract Length(years),Contract Category
15494,2020 ~ 2022,2020,2022,2.0,Contract
1693,2015 ~ 2020,2015,2020,5.0,Contract
14205,2018 ~ 2021,2018,2021,3.0,Contract


# Height

In [26]:
fifa['Height'].dtype

dtype('O')

In [23]:
fifa['Height'].unique()

In [28]:
def convert_height(height):
    if 'cm' in height:
        return int(height.strip('cm'))
    else:
        feet,inches = height.split("'")
        calculate_inches = int(feet)*12 + int(inches.strip('"'))
        calculate_cm = round(calculate_inches*2.54)
        return calculate_cm
        
#apply fn to column Height

fifa['Height']= fifa['Height'].apply(convert_height)

In [22]:
fifa= fifa.rename(columns= {'Height':"Height(cm)"})

fifa['Height(cm)'].unique()

## Weight

In [43]:
fifa['Weight'].dtype

dtype('O')

In [21]:
fifa['Weight'].unique()

In [45]:
def convert_weight(weight):
    if "kg" in weight:
        return int(weight.strip('kg'))
    else:
        lbs_to_kg = int(weight.strip('lbs'))/2.2
        return round(lbs_to_kg)

#apply fn to column data
fifa['Weight'] = fifa['Weight'].apply(convert_weight)
    

In [46]:
fifa = fifa.rename(columns= {'Weight': 'Weight(kg)'})
fifa['Weight(kg)'].unique()

array([ 72,  83,  87,  70,  68,  80,  71,  91,  73,  85,  92,  69,  84,
        96,  81,  82,  75,  86,  89,  74,  76,  64,  78,  90,  66,  60,
        94,  79,  67,  65,  59,  61,  93,  88,  97,  77,  62,  63,  95,
       100,  58,  98, 103,  99, 102,  56, 101,  57,  55, 104, 107, 110,
        53,  50,  54,  52], dtype=int64)

## Loan Date End

In [12]:
fifa['Loan Date End'].dtype

dtype('O')

In [13]:
fifa['Loan Date End'].unique()

array([nan, '30-Jun-21', '31-Dec-20', '30-Jan-21', '30-Jun-22',
       '31-May-21', '05-Jul-21', '31-Dec-21', '01-Jul-21', '01-Jan-21',
       '31-Aug-21', '31-Jan-21', '30-Dec-21', '23-Jun-21', '03-Jan-21',
       '27-Nov-21', '17-Jan-21', '30-Jun-23', '31-Jul-21', '22-Nov-20',
       '31-May-22', '30-Dec-20', '04-Jan-21', '30-Nov-20', '01-Aug-21'],
      dtype=object)

In [26]:
date1= pd.to_datetime(fifa['Loan Date End'],errors= 'coerce', format= '%d-%b-%y')
date2= pd.to_datetime(fifa['Loan Date End'],errors= 'coerce', format= '%d-%B-%y')
fifa['Loan Date End']= date1.fillna(date2)

In [17]:
on_loan_contracts = fifa[fifa['Contract'].str.contains('On Loan')]
on_loan_contracts[['Contract','Contract Category','Loan Date End']]

KeyError: "['Contract Category'] not in index"

## W/F

In [28]:
fifa['W/F'].dtype

dtype('O')

In [29]:
fifa['W/F'].unique()

array(['4 ★', '3 ★', '5 ★', '2 ★', '1 ★'], dtype=object)

In [30]:
fifa['W/F']= fifa['W/F'].str.strip('★')
fifa['W/F'].unique()

array(['4 ', '3 ', '5 ', '2 ', '1 '], dtype=object)

In [31]:
fifa['IR']= fifa['IR'].str.strip('★')
fifa['IR'].unique()

array(['5 ', '3 ', '4 ', '2 ', '1 '], dtype=object)

In [32]:
fifa['SM']= fifa['SM'].str.strip('★')
fifa['SM'].unique()

array(['4', '5', '1', '2', '3'], dtype=object)

In [77]:
fifa['Wage'].unique()

array(['€560K', '€220K', '€125K', '€370K', '€270K', '€240K', '€250K',
       '€160K', '€260K', '€210K', '€310K', '€130K', '€350K', '€300K',
       '€190K', '€145K', '€195K', '€100K', '€140K', '€290K', '€82K',
       '€110K', '€230K', '€155K', '€200K', '€165K', '€95K', '€170K',
       '€105K', '€115K', '€150K', '€135K', '€55K', '€58K', '€81K', '€34K',
       '€120K', '€59K', '€90K', '€65K', '€56K', '€71K', '€18K', '€75K',
       '€47K', '€20K', '€84K', '€86K', '€74K', '€78K', '€27K', '€68K',
       '€85K', '€25K', '€46K', '€83K', '€54K', '€79K', '€175K', '€43K',
       '€49K', '€45K', '€38K', '€41K', '€39K', '€23K', '€51K', '€50K',
       '€87K', '€30K', '€14K', '€69K', '€31K', '€64K', '€53K', '€35K',
       '€21K', '€28K', '€17K', '€33K', '€70K', '€32K', '€89K', '€26K',
       '€40K', '€76K', '€72K', '€48K', '€36K', '€29K', '€60K', '€16K',
       '€37K', '€24K', '€52K', '€0', '€62K', '€73K', '€63K', '€19K',
       '€1K', '€66K', '€80K', '€12K', '€2K', '€42K', '€13K', '€900',
       '€5

In [41]:
#for wage in fifa['Wage']:
    fifa['Wage']  = wage.strip("€")

In [47]:
#for i, wage in enumerate(fifa['Wage']):
    fifa.loc[i, 'Wage'] = wage.strip('€')  # Corrected: Assign the result back to 'fifa['Wage']'


In [46]:
#for i, wage in enumerate(fifa['Wage']):
    fifa.loc[i, 'Wage'] = wage.strip("€")

# Now let's check a few random samples from the updated 'Wage' column:
sample_wages = fifa['Wage'].sample(3)
sample_wages


IndentationError: unexpected indent (3450788593.py, line 2)

In [73]:
#for wage in fifa['Wage']:
    fifa['Wage']= wage.replace('€','')

fifa['Wage'].sample()

1020    1K
Name: Wage, dtype: object

In [37]:
fifa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID                18979 non-null  int64         
 1   Name              18979 non-null  object        
 2   LongName          18979 non-null  object        
 3   photoUrl          18979 non-null  object        
 4   playerUrl         18979 non-null  object        
 5   Nationality       18979 non-null  object        
 6   Age               18979 non-null  int64         
 7   ↓OVA              18979 non-null  int64         
 8   POT               18979 non-null  int64         
 9   Club              18979 non-null  object        
 10  Contract          18979 non-null  object        
 11  Positions         18979 non-null  object        
 12  Height            18979 non-null  object        
 13  Weight            18979 non-null  object        
 14  Preferred Foot    1897

In [33]:
for ir in fifa['IR']:
    fifa['IR']= int(ir)

In [34]:
for sm in fifa['SM']:
    fifa['SM']= int(sm)

In [36]:
for wf in fifa['W/F']:
    fifa['W/F']= int(wf)

In [24]:
fifa['Joined']= pd.to_datetime(fifa['Joined'], format= '%d-%B-%y')

In [ ]:
fifa[]